In [1]:
#Ryan Galloway
#CS5396
#April 2nd, 2021

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import mysql.connector

noPages = 2

def getData(pageNo): 
    #setting user agaent header to try and fool Amazon so it doesn't know we are scraping (not working) when using request
    #https://www.codementor.io/blog/python-web-scraping-63l2v9sf2q   ==> Header Inspection
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    #Creates request object from the given url and parameters
    url = requests.get('https://www.amazon.com/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)
    
    #set content varaible = to content of the url request object (html code scraped)
    content = url.content
    
    #create a BeautifulSoup object using the content (soupify it)
    soup = BeautifulSoup(content)
    #print(soup.prettify)
    
    #Holds the data we are scraping for all pages
    scrapedData = []
    
    #Parses the HTML content from the "parent tag" (tag that holds all the data about the books)
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        
        #each of these variables holds the attribute value of the item we are looking for
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        usersRated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        #temp list to hold data scraped for each iteration of scraping
        temp=[]
        
        #These are checking the objects returned from the .find BS function call
        #which is why is not None is used, to test if the variable 
        #is valid. If an object has no value associated
        #set a default value.
        if name is not None:
            temp.append(n[0]['alt'])
        else:
            temp.append("unknown-product")
            
        if author is not None:
            #print(author.text)
            temp.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                temp.append(author.text)
            else:    
                temp.append('0')
                
        if rating is not None:
            #print(rating.text)
            temp.append(rating.text)
        else:
            temp.append('-1')

        if usersRated is not None:
            #print(price.text)
            temp.append(usersRated.text)
        else:
            temp.append('0')     

        if price is not None:
            #print(price.text)
            temp.append(price.text)
        else:
            temp.append('0')
        
        #master list of scraped data
        scrapedData.append(temp)    
    return scrapedData

#List to hold returned data scraped
results = []
for i in range(1, noPages+1):
    results.append(getData(i))
#print (results[0], 'alt')
    
#initialize flatten: This variable will flatten the results of separate sublists
#when we create the data frame using the results list
flatten = lambda l: [item for sublist in l for item in sublist]

#create the data frame
dataFrame = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
#write data frame to csv file
dataFrame.to_csv('amazonBooks.csv', index=False, encoding='utf-8')

#read the csv for processing
dataFrame = pd.read_csv("amazonBooks.csv")


#Remove out of 5 from ratings column, lamba expression splits the string after the 
#rating
dataFrame['Rating'] = dataFrame['Rating'].apply(lambda x: x.split()[0])

#convert rating to numeric from string 
dataFrame['Rating'] = pd.to_numeric(dataFrame['Rating'])

#remove $ from price
dataFrame["Price"] = dataFrame["Price"].str.replace('$', ' ', regex = True)


#turn price into float
dataFrame["Price"] = dataFrame["Price"].astype(float)

#Remove comma from number of ratings
dataFrame["Customers_Rated"] = dataFrame["Customers_Rated"].str.replace(',', '', regex = True)

#convert rating string to numeric, 
dataFrame["Customers_Rated"] = pd.to_numeric(dataFrame["Customers_Rated"], errors = 'ignore')

dataFrame.dtypes

dataFrame.to_csv('AmznBooksTransformed.csv', index=False, encoding='utf-8')
